<a href="https://colab.research.google.com/github/vishnucramesh/deepfake-detection/blob/master/DeepfakeDetector_DenseNet_TF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Step 1: Upload the API key file provided by kaggle to a location in google drive
Step 2: Set the config file as the environemnt variable
Step 5: Change working directory
Step 3: Get kaggle download link from kaggle 
"""

import os

!pip install kaggle


PATH = '/Users/vishnu/Work/uni/VISOPE/deepfake-image-detector/dataset'

# set kaggle config file directory
os.environ['KAGGLE_CONFIG_DIR'] = PATH

%cd $PATH

!kaggle datasets download -d xhlulu/140k-real-and-fake-faces --unzip

In [5]:
pip install matplotlib pillow sklearn opencv-python tqdm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.6/500.6 kB 6.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import cv2
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf
from tqdm import tqdm

In [8]:
def build_model(pretrained):
    model = Sequential([
        pretrained,
        layers.GlobalAveragePooling2D(),
        layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    
    return model

In [11]:
base_path = '/Users/vishnu/Work/uni/VISOPE/deepfake-image-detector/dataset/real_vs_fake/real-vs-fake/'
image_gen = ImageDataGenerator(rescale=1./255.)
batch_size = 64
train_flow = image_gen.flow_from_directory(
    base_path + 'train/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)
valid_flow = image_gen.flow_from_directory(
    base_path + 'valid/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)
test_flow = image_gen.flow_from_directory(
    base_path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [9]:
densenet = DenseNet121(
    include_top=False,
    input_shape=(224,224,3)
)
model = build_model(densenet)
model.summary()

2022-09-14 17:13:17.378660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


29084464/29084464 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 1025      
                                                                 
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
train_steps = 1000//64
valid_steps = 200//64

earlystop_callback = EarlyStopping(monitor='loss', patience=3)
checkpoint_callback = ModelCheckpoint(filepath=f"model.hdf5", 
                             monitor='val_loss',
                             verbose=1, 
                             save_best_only=True,
                             mode='min')

history = model.fit(
      train_flow,
      epochs=10,
      steps_per_epoch = train_steps,
      validation_data = valid_flow,
      validation_steps = valid_steps,
      callbacks=[earlystop_callback, checkpoint_callback]
)

Epoch 1/10
15/15 [==============================] - ETA: 0s - loss: 0.4977 - accuracy: 0.7729 
Epoch 1: val_loss improved from inf to 11.73476, saving model to model.hdf5
15/15 [==============================] - 302s 20s/step - loss: 0.4977 - accuracy: 0.7729 - val_loss: 11.7348 - val_accuracy: 0.4844
Epoch 2/10
15/15 [==============================] - ETA: 0s - loss: 0.4102 - accuracy: 0.8115 
Epoch 2: val_loss improved from 11.73476 to 1.94965, saving model to model.hdf5
15/15 [==============================] - 318s 21s/step - loss: 0.4102 - accuracy: 0.8115 - val_loss: 1.9497 - val_accuracy: 0.4583
Epoch 3/10
15/15 [==============================] - ETA: 0s - loss: 0.4238 - accuracy: 0.8167 
Epoch 3: val_loss did not improve from 1.94965
15/15 [==============================] - 314s 21s/step - loss: 0.4238 - accuracy: 0.8167 - val_loss: 2.8346 - val_accuracy: 0.4896
Epoch 4/10
15/15 [==============================] - ETA: 0s - loss: 0.2988 - accuracy: 0.8760 
Epoch 4: val_loss did n

In [ ]:
y_pred = model.predict_generator(test_flow)
y_test = test_flow.classes

In [ ]:
print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))
print("AP Score:", metrics.average_precision_score(y_test, y_pred))
print()
print(metrics.classification_report(y_test, y_pred > 0.5))

In [ ]:
def plot_loss(epochs, loss, val_loss):
    plt.plot(epochs, loss, 'bo', label='Training Loss')
    plt.plot(epochs, val_loss, 'orange', label = 'Validation Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

def plot_accuracy(epochs, acc, val_acc):
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'orange', label = 'Validation accuracy')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plot_loss(range(1, len(loss) + 1), loss, val_loss)
plot_accuracy(range(1, len(loss) + 1), acc, val_acc)

In [ ]:
y_pred = model.predict(test_flow)
y_test = test_flow.classes

In [ ]:
from sklearn import metrics

print("ROC AUC Score:", metrics.roc_auc_score(y_test, y_pred))

print("AP Score:", metrics.average_precision_score(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred > 0.5))
